In [20]:
import time
import numpy as np
import pandas as pd
# import seaborn as sns
import string
import re
# sns.set()

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings("ignore")

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [21]:
# use either top model-aware or model-agnostic
# df = pd.read_json('../SHROOM_dev-v2_validation/val.model-agnostic.json')
df = pd.read_json('data/val.model-aware.v2.json')
df["label"] = df["label"].map({"Hallucination":1, "Not Hallucination":0})

df = df.dropna(how="any", axis=1)
df.head()

print(df['label'].value_counts())

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,A sloping top .,tgt,The sides of the casket were covered with heav...,A decorative feature that sits on top of somet...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",1,0.6
1,To react too much .,tgt,Please try not to overreact if she drives badl...,To react too much or too intensely .,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
2,The process of spoiling ; the state of being s...,tgt,"To prevent spoilage , store in a cool , dry pl...",The process of spoiling .,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",1,0.6
3,To arrange in a particular way .,tgt,The way the opposition has framed the argument...,To construct in words so as to establish a con...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",1,0.6
4,A feeling of concern ; a feeling of anxiety .,tgt,To mix with thy concernments i desist . What i...,That in which one is concerned or interested ;...,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",1,0.6


In [22]:
import nltk
# nltk.download('wordnet')
# nltk.download('omw-1.4')
# nltk.download('stopwords')

In [23]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
# stemmer = nltk.SnowballStemmer("english")



def clean_text(text):
    '''
    Make text lowercase, remove text in square brackets, remove links, remove punctuation,
    and remove words containing numbers.
    '''
    text = str(text).lower()
    
    # Remove text in square brackets
    text = re.sub(r'\[.*?\]', '', text)
    
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove punctuation
    text = re.sub(rf'[{re.escape(string.punctuation)}]', ' ', text)
    
    # Remove new lines
    text = re.sub(r'\n', ' ', text)
    
    # Remove words containing numbers
    text = re.sub(r'\w*\d\w*', '', text)
    
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def preprocess_data(text):
    text = clean_text(text)  # Clean punctuation, URLs, and so on
    
    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stop_words)
    
    # Lemmatize all the words in the sentence
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())
    
    return text

# Example usage
example_text = "This is an example text with a URL: http://example.com and an email: example@example.com."
cleaned_text = preprocess_data(example_text)
print(cleaned_text)


example text url email example example com


In [24]:
example_preprocessed = preprocess_data("A decorative feature that sits on top of something .")
example_preprocessed

'decorative feature sits top something'

In [25]:
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    df[x] = df[x].apply(preprocess_data)

# X = np.reshape(X,(499))
# X.shape, y.shape
# X = df.drop(['label','labels', 'model', 'p(Hallucination)','task','ref'], axis=1)
# y = df['label']
df

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination)
0,sloping top,tgt,side casket covered heavy black broadcloth vel...,decorative feature sits top something,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Not Halluci...",1,0.6
1,react much,tgt,please try overreact drive badly first learnin...,react much intensely,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
2,process spoiling state spoilt,tgt,prevent spoilage store cool dry place meaning ...,process spoiling,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Hallucinati...",1,0.6
3,arrange particular way,tgt,way opposition framed argument make hard u win...,construct word establish context understanding...,ltg/flan-t5-definition-en-base,DM,"[Hallucination, Not Hallucination, Not Halluci...",1,0.6
4,feeling concern feeling anxiety,tgt,mix thy concernments desist meaning concernment,one concerned interested concern affair interest,ltg/flan-t5-definition-en-base,DM,"[Not Hallucination, Hallucination, Hallucinati...",1,0.6
...,...,...,...,...,...,...,...,...,...
496,using gas fired device way stop people using n...,either,doonii fayyadamuun meeshaa geejibuun namootaba...,using ship transport good far efficient way mo...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
497,since montevideo located south equator summer ...,either,وبما أن مونتيفيديو موجودة في جنوب خط الاستواء،...,since montevideo south equator summer winter n...,facebook/nllb-200-distilled-600M,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
498,also supporter singapore deputy prime minister...,either,gin abiabi hiya han deputy prime minister han ...,greeted singapore deputy prime minister wong k...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Not Hallucinati...",1,0.8
499,concept worm parasitic organism trained behave...,either,འབུ་ཞེས་པའི་ཐ་སྙད་དེ་ཉིད་འབུ་སྲིན་དཔྱད་རིག་པ་བ...,term bug used entomologist formal sense group ...,facebook/nllb-200-distilled-600M,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0


In [26]:
# extract preprocessed data as CSV file,
# also i extract them based on tasks, so that they can be fed separately
df = df.drop(['labels'], axis=1)
dm_df = df[df['task'] == 'DM']
mt_df = df[df['task'] == 'MT']
pg_df = df[df['task'] == 'PG']


In [27]:
len(dm_df),len(mt_df), len(pg_df)

(188, 188, 125)

In [28]:
number_of_na = pg_df.isna().sum()
number_of_na

hyp                 0
ref                 0
src                 0
tgt                 0
model               0
task                0
label               0
p(Hallucination)    0
dtype: int64

In [29]:
import pathlib
# file_path = '/Users/byun/Desktop/School/semeval/eda_nlp/thesis_data_before/'
file_path = r"C:\Users\marko\OneDrive\바탕 화면\semeval\Task 6 - windows\eda_nlp-20240601T173748Z-001"
file_path_uni = pathlib.Path(file_path).as_posix()
print(file_path_uni)
# Save the DataFrame as a CSV file at the specified location
dm_df.to_csv(file_path_uni + '/dm_df.csv', index=False)
mt_df.to_csv(file_path_uni + '/mt_df.csv', index=False)
pg_df.to_csv(file_path_uni + '/pg_df.csv', index=False)

C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001


In [9]:
eda_path = r"C:\Users\marko\OneDrive\바탕 화면\semeval\Task 6 - windows\eda_nlp-20240601T173748Z-001\eda_nlp"
eda_path_posix = pathlib.Path(eda_path).as_posix()
eda_path_posix

'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp'

In [9]:
# C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/thesis_data_after
# task PG, num_aug 16
pg_rd_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/pg_df_rd.csv'
pg_ri_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/pg_df_ri.csv'
pg_rs_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/pg_df_rs.csv'
pg_sr_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/pg_df_sr.csv'

pg_rd = pd.read_csv(pg_rd_path)
pg_ri = pd.read_csv(pg_ri_path)
pg_rs = pd.read_csv(pg_rs_path)
pg_sr = pd.read_csv(pg_sr_path)

# num_aug 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/pg_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/pg_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/pg_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/pg_df_sr.csv'

pg_rd_160 = pd.read_csv(path1)
pg_ri_160 = pd.read_csv(path2)
pg_rs_160 = pd.read_csv(path3)
pg_sr_160 = pd.read_csv(path4)

In [10]:
# task DM 16
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/dm_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/dm_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/dm_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/dm_df_sr.csv'

dm_rd = pd.read_csv(path1)
dm_ri = pd.read_csv(path2)
dm_rs = pd.read_csv(path3)
dm_sr = pd.read_csv(path4)

# 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rd/dm_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/ri/dm_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rs/dm_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/sr/dm_df_sr.csv'

dm_rd_160 = pd.read_csv(path1)
dm_ri_160 = pd.read_csv(path2)
dm_rs_160 = pd.read_csv(path3)
dm_sr_160 = pd.read_csv(path4)

In [11]:
# task MT 16
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/mt_df_sr.csv'

mt_rd = pd.read_csv(path1)
mt_ri = pd.read_csv(path2)
mt_rs = pd.read_csv(path3)
mt_sr = pd.read_csv(path4)

# 160
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after_160/sr/mt_df_sr.csv'

mt_rd_160 = pd.read_csv(path1)
mt_ri_160 = pd.read_csv(path2)
mt_rs_160 = pd.read_csv(path3)
mt_sr_160 = pd.read_csv(path4)

In [12]:
#
path1 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rd/mt_df_rd.csv'
path2 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/ri/mt_df_ri.csv'
path3 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/rs/mt_df_rs.csv'
path4 = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/eda_nlp-20240601T173748Z-001/eda_nlp/thesis_data_after/sr/mt_df_sr.csv'

mt_rd = pd.read_csv(path1)
mt_ri = pd.read_csv(path2)
mt_rs = pd.read_csv(path3)
mt_sr = pd.read_csv(path4)

In [13]:
# pg_rd = pg_rd.dropna(how="any", axis=1)
pg_rd = pg_rd.fillna('')
pg_ri = pg_ri.fillna('')
pg_rs = pg_rs.fillna('')
pg_sr = pg_sr.fillna('')

dm_rd = dm_rd.fillna('')
dm_ri = dm_ri.fillna('')
dm_rs = dm_rs.fillna('')
dm_sr = dm_sr.fillna('')

mt_rd = mt_rd.fillna('')
mt_ri = mt_ri.fillna('')
mt_rs = mt_rs.fillna('')
mt_sr = mt_sr.fillna('')

pg_rd_160 = pg_rd_160.fillna('')
pg_ri_160 = pg_ri_160.fillna('')
pg_rs_160 = pg_rs_160.fillna('')
pg_sr_160 = pg_sr_160.fillna('')

dm_rd_160 = dm_rd_160.fillna('')
dm_ri_160 = dm_ri_160.fillna('')
dm_rs_160 = dm_rs_160.fillna('')
dm_sr_160 = dm_sr_160.fillna('')

mt_rd_160 = mt_rd_160.fillna('')
mt_ri_160 = mt_ri_160.fillna('')
mt_rs_160 = mt_rs_160.fillna('')
mt_sr_160 = mt_sr_160.fillna('')
# drop na just deleted the model column i think
mt_rd_160

,hyp,ref,src,tgt,model,label,p(Hallucination),Unnamed: 7
0,john john one,either,,john ann like,,1,1.0,
1,john john one,either,,john ann like,,1,1.0,
2,john john one,either,,john ann like,,1,1.0,
3,john one,either,,john ann,,1,1.0,
4,john john one,either,,john ann like,,1,1.0,
...,...,...,...,...,...,...,...,...
7849,dont money buy dictionary,either,,money buy dictionary,,0,0.0,
7850,dont money buy dictionary,either,,buy dictionary,,0,0.0,
7851,dont money buy dictionary,either,,money buy dictionary,,0,0.0,
7852,dont money buy dictionary,either,,money buy dictionary,,0,0.0,


In [14]:
mt_sr_160

,hyp,ref,src,tgt,model,label,p(Hallucination),Unnamed: 7
0,saint john the apostle saint john the apostle one,either,,trick ann like,,1,1.0,
1,john john,either,,st john the apostle ann like,,1,1.0,
2,john john unitary,either,,st john ann like,,1,1.0,
3,john john unitary,either,,whoremonger ann like,,1,1.0,
4,john john peerless,either,,john the divine ann like,,1,1.0,
...,...,...,...,...,...,...,...,...
7849,dont money bargain dictionary,either,,money buy lexicon,,0,0.0,
7850,dont money grease ones palms dictionary,either,,money purchase dictionary,,0,0.0,
7851,dont money purchase dictionary,either,,money buy lexicon,,0,0.0,
7852,dont money buy lexicon,either,,money bargain dictionary,,0,0.0,


In [15]:
mt_sr

,hyp,ref,src,tgt,model,label,p(Hallucination),Unnamed: 7
0,john lackland john lackland one,either,,john ann like,,1,1.0,
1,john john one and only,either,,john ann like,,1,1.0,
2,whoremaster whoremaster one,either,,john ann like,,1,1.0,
3,john john matchless,either,,john ann like,,1,1.0,
4,gospel according to john gospel according to j...,either,,john ann like,,1,1.0,
...,...,...,...,...,...,...,...,...
1117,dont money buy lexicon,either,,money buy dictionary,,0,0.0,
1118,dont money corrupt dictionary,either,,money buy dictionary,,0,0.0,
1119,dont money grease ones palms dictionary,either,,money buy dictionary,,0,0.0,
1120,dont money buy lexicon,either,,money buy dictionary,,0,0.0,


In [16]:
# original dataset before data aug
# X, y = pg_df['hyp'], pg_df['label']
# X, y = dm_df['hyp'], dm_df['label']
# X, y = mt_df['hyp'], mt_df['label']

# task PG
# X, y = pg_rd[['hyp','tgt']] , pg_rd['label']
# X, y = pg_ri[['hyp','tgt']] , pg_ri['label']
# X, y = pg_rs[['hyp','tgt']] , pg_rs['label']
# X, y = pg_sr[['hyp','tgt']] , pg_sr['label']
# task DM
# X, y = dm_rd[['hyp','tgt']] , dm_rd['label']
# X, y = dm_ri[['hyp','tgt']], dm_ri['label']
# X, y = dm_rs[['hyp','tgt']] , dm_rs['label']
# X, y = dm_sr[['hyp','tgt']] , dm_sr['label']
# task MT
# X, y = mt_rd[['hyp','tgt']] , mt_rd['label']
# X, y = mt_ri[['hyp','tgt']] , mt_ri['label']
# X, y = mt_rs[['hyp','tgt']] , mt_rs['label']
X, y = mt_sr[['hyp','tgt']] , mt_sr['label']

# X, y = df['hyp'], df['label']
# one way of getting label where task is DM, df.loc[df['task'] == 'DM', 'label']
# X, y = df[df['task'] == 'PG']['hyp'], df[df['task'] == 'PG']['label']

# task PG
# X, y = pg_rd_160['hyp'] , pg_rd_160['label']
# X, y = pg_ri_160['hyp'] , pg_ri_160['label']
# X, y = pg_rs_160['hyp'] , pg_rs_160['label']
# X, y = pg_sr_160['hyp'] , pg_sr_160['label']
# task DM
# X, y = dm_rd_160['hyp'] , dm_rd_160['label']
# X, y = dm_ri_160['hyp'] , dm_ri_160['label']
# X, y = dm_rs_160['hyp'] , dm_rs_160['label']
# X, y = dm_sr_160['hyp'] , dm_sr_160['label']
# task MT
# X, y = mt_rd_160['hyp'] , mt_rd_160['label']
# X, y = mt_ri_160['hyp'] , mt_ri_160['label']
# X, y = mt_rs_160['hyp'] , mt_rs_160['label']
# X, y = mt_sr_160['hyp'] , mt_sr_160['label']

In [17]:
print("Class distribution:\n", y.value_counts())

Class distribution:
 label
0    636
1    486
Name: count, dtype: int64


In [18]:
X = np.array(X)

y = np.array(y)


In [19]:
for i in range(len(X)):
    X[i] = str(X[i])

In [20]:
# split into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,  random_state=42)
len(X_train), len(y_train), len(X_test), len(y_test)

(897, 897, 225, 225)

In [21]:
# count number of none 0 or 1s
number_of_na = df.isna().sum()
print("Number of na:", number_of_na)

Number of na: hyp                 0
ref                 0
src                 0
tgt                 0
model               0
task                0
label               0
p(Hallucination)    0
dtype: int64


In [22]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score

In [23]:
# paths to final test data
final_ag_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'
final_aw_path = 'C:/Users/marko/OneDrive/바탕 화면/semeval/Task 6 - windows/final_test_data/test.model-agnostic.json'

final_ag_df = pd.read_json(final_ag_path)
final_aw_df = pd.read_json(final_aw_path)
final_ag_df["label"] = final_ag_df["label"].map({"Hallucination":1, "Not Hallucination":0})
final_aw_df["label"] = final_aw_df["label"].map({"Hallucination":1, "Not Hallucination":0})
# perform preprocessing on final_df AND split it based on tasks!
columns = ['hyp','src','tgt'] # exclude task
for x in columns:
    final_ag_df[x] = final_ag_df[x].apply(preprocess_data)
for x in columns:
    final_aw_df[x] = final_aw_df[x].apply(preprocess_data)
final_ag_pg = final_ag_df[final_ag_df['task'] == 'PG']
final_ag_dm = final_ag_df[final_ag_df['task'] == 'DM']
final_ag_mt = final_ag_df[final_ag_df['task'] == 'MT']

final_aw_pg = final_aw_df[final_aw_df['task'] == 'PG']
final_aw_dm = final_aw_df[final_aw_df['task'] == 'DM']
final_aw_mt = final_aw_df[final_aw_df['task'] == 'MT']

final_ag_df

,id,src,tgt,hyp,task,labels,label,p(Hallucination)
0,1,ты удивишься если я скажу что на самом деле ме...,would surprised told name isnt actually tom,youre gonna surprised say real name isnt tom,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0
1,2,еды будет полно,plenty food,food full,MT,"[Hallucination, Not Hallucination, Hallucinati...",1,0.8
2,3,думаете том будет меня ждать,think tom wait,think toms gonna wait,MT,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.2
3,6,два брата довольно разные,two brothers pretty different,theres lot friends,MT,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
4,7,infradiaphragmatic intra suprasellar craniopha...,medicine diaphragm,anatomy relating diaphragm,DM,"[Hallucination, Hallucination, Hallucination, ...",1,0.8
...,...,...,...,...,...,...,...,...
1495,2992,я никогда не говорил мэри что чувствую,never told mary felt,ive never told mary feel,MT,"[Hallucination, Not Hallucination, Not Halluci...",0,0.4
1496,2993,beat rat tailed kyoodle runs steers eric laid ...,mutt dog mixed breed little value noisy dog,slang mustang,DM,"[Hallucination, Hallucination, Hallucination, ...",1,1.0
1497,2994,ты знаешь почему они прекратили говорить,know stopped talking,know stopped talking,MT,"[Hallucination, Not Hallucination, Not Halluci...",0,0.4
1498,2996,anyone back,anyone confirm,anyone corroborate,PG,"[Not Hallucination, Not Hallucination, Not Hal...",0,0.0


In [24]:
from sklearn.model_selection import cross_val_score

In [25]:
import numpy as np
from sklearn.model_selection import StratifiedKFold
from transformers import TFBertForSequenceClassification, BertTokenizer, AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.optimizers import Adam
# two other models i wanna try is BERT-large and RoBERTa
checkpoint = 'bert-base-uncased'
# checkpoint = 'bert-large-uncased'
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained(checkpoint)


In [26]:
import tensorflow as tf
import transformers
print(tf.__version__)
print(transformers.__version__)

2.16.1
4.41.2


In [27]:
# import os
# os.environ['TF_USE_LEGACY_KERAS'] = '1' 

In [28]:
import tf_keras
adam = tf_keras.src.optimizers.Adam(learning_rate=1e-6)

In [29]:
# this here uses manual cross validation!
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import datasets
import time

BATCH_SIZE = 8
EPOCHS = 5
N_SPLITS = 5

# Shuffle and split data into N_SPLITS folds
indices = np.arange(len(X))
np.random.shuffle(indices)
folds = np.array_split(indices, N_SPLITS)

# Initialize lists to store evaluation metrics
all_val_preds = []
accuracies = []
precisions = []
recalls = []
f1s = []

def tokenize_function(example):
    return tokenizer(example['hyp'], text_pair=example['tgt'], truncation=True, padding='max_length', max_length=100)

def order(inp):
    data = list(inp.values())
    return {
        'input_ids': tf.convert_to_tensor(data[1], dtype=tf.int32),
        'token_type_ids': tf.convert_to_tensor(data[2], dtype=tf.int32),
        'attention_mask': tf.convert_to_tensor(data[3], dtype=tf.int32),
    }, data[0]
# changed from optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-5), to without legacy
# bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1)
# bert_model.compile(
#     # optimizer=Adam,
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics=['accuracy']
# )

for i in range(N_SPLITS):
    # Reset model for each fold
    bert_model = TFBertForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
    
    # Compile the model with appropriate loss function and optimizer
    bert_model.compile(
        # first try out lr of 1e-6, then 1e-5, then 1e-4
        # optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=1e-6),
        optimizer=adam,
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy']
    )
    # the code below resets model, and I might try it out? maybe idk
    # Prepare training and validation indices
    val_indices = folds[i]
    train_indices = np.hstack([folds[j] for j in range(N_SPLITS) if j != i])

    # Split the data
    X_train2, X_val = X[train_indices], X[val_indices]
    y_train2, y_val = y[train_indices], y[val_indices]
    # Create Pandas DataFrames
    X_train2_df = pd.DataFrame({'hyp': X_train2[:,0], 'tgt':X_train2[:,1], 'label': y_train2})
    X_val_df = pd.DataFrame({'hyp': X_val[:,0], 'tgt': X_val[:,1], 'label': y_val})

    # Convert to HuggingFace Datasets
    df_train = datasets.Dataset.from_pandas(X_train2_df)
    df_val = datasets.Dataset.from_pandas(X_val_df)
    dataset = datasets.DatasetDict({'train': df_train, 'val': df_val})

    # Tokenize the datasets
    tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp','tgt'])

    # Convert to TensorFlow Datasets
    train_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['train'][:])
    train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
    train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    val_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['val'][:])
    val_dataset = val_dataset.batch(BATCH_SIZE)
    val_dataset = val_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

    # Train the model
    start = time.time()
    bert_model.fit(train_dataset, validation_data=val_dataset, epochs=EPOCHS)
    end = time.time()
    elapsed_time = round(end - start, 2)
    print(f'Fitting fold {i+1} is DONE!! Time: {elapsed_time}s')

    # Evaluate the model
    evaluated = bert_model.evaluate(val_dataset)
    val_preds = bert_model.predict(val_dataset).logits
#     val_preds = tf.nn.softmax(val_preds, axis=1).numpy()[:, 1]  # Probability of class 1, 1 try
    # val_preds = tf.nn.sigmoid(val_preds).numpy().squeeze()[:, 1] # 2nd attempt, also i might need try out
    # using a [:, 0] instead?
    # Apply sigmoid activation
    val_preds = tf.nn.sigmoid(val_preds).numpy()

    # Check if val_preds has more than one dimension
    if val_preds.ndim > 1:
        val_preds = val_preds.squeeze()
        # Ensure that there are at least two dimensions to index
        if val_preds.ndim > 1:
            val_preds = val_preds[:, 1]
        else:
            # Handle case where squeezing resulted in a single dimension
            val_preds = val_preds.flatten()
    else:
        val_preds = val_preds.flatten()
    precision, recall, f1, _ = precision_recall_fscore_support(y_val, val_preds.round(), average='weighted')
    acc = accuracy_score(y_val, val_preds.round())

    all_val_preds.append(val_preds)
    accuracies.append(acc)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)

# Calculate and print average metrics
print(f'Mean Accuracy: {np.mean(accuracies)}')
print(f'Mean Precision: {np.mean(precisions)}')
print(f'Mean Recall: {np.mean(recalls)}')
print(f'Mean F1 Score: {np.mean(f1s)}')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map:   0%|          | 0/225 [00:00<?, ? examples/s]Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Map: 100%|██████████| 225/225 [00:00<00:00, 2019.39 examples/s]


Epoch 1/5
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert



  3/113 [..............................] - ETA: 2:33 - loss: 0.6974 - accuracy: 0.7083 

KeyboardInterrupt: 

In [ ]:
# Convert to TensorFlow Datasets
def order_final(example):
    return {
        'input_ids': tf.convert_to_tensor(example['input_ids'], dtype=tf.int32),
        'attention_mask': tf.convert_to_tensor(example['attention_mask'], dtype=tf.int32),
        'token_type_ids': tf.convert_to_tensor(example['token_type_ids'], dtype=tf.int32),
    }, tf.convert_to_tensor(example['label'], dtype=tf.int32)


# final two datasets
ag_X, ag_y = final_ag_pg[['hyp','tgt']], final_ag_pg['label']
# ag_X, ag_y = final_ag_dm[['hyp','tgt']], final_ag_dm['label']
# ag_X, ag_y = final_ag_mt[['hyp','tgt']], final_ag_mt['label']

# ag_X, ag_y = final_aw_pg['hyp'], final_aw_pg['label']
# ag_X, ag_y = final_aw_dm['hyp'], final_aw_dm['label']
# ag_X, ag_y = final_aw_mt['hyp'], final_aw_mt['label']

# Create Pandas DataFrames
final_df = pd.DataFrame({'hyp': ag_X['hyp'], 'tgt': ag_X['tgt'], 'label': ag_y})

# Convert to HuggingFace Datasets
final_df2 = datasets.Dataset.from_pandas(final_df)
dataset = datasets.DatasetDict({'final': final_df2})

# Tokenize the datasets
tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp','tgt'])

# Convert to TensorFlow Datasets
final_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['final'][:])
final_dataset = final_dataset.batch(BATCH_SIZE)
final_dataset = final_dataset.map(order_final, num_parallel_calls=tf.data.AUTOTUNE)


Map: 100%|██████████| 562/562 [00:00<00:00, 3447.44 examples/s]


In [ ]:
# my attempt failed for final dataset
# evaluated = model.evaluate(final_dataset)
final_preds = bert_model.predict(final_dataset).logits
final_preds = tf.nn.sigmoid(final_preds).numpy()
# Check if val_preds has more than one dimension
if final_preds.ndim > 1:
    final_preds = final_preds.squeeze()
    # Ensure that there are at least two dimensions to index
    if final_preds.ndim > 1:
        final_preds = final_preds[:, 1]
    else:
        # Handle case where squeezing resulted in a single dimension
        final_preds = final_preds.flatten()
else:
    final_preds = final_preds.flatten()


71/71 [==============================] - 43s 599ms/step


In [ ]:
final_preds_binary = [1 if x > 0.5 else 0 for x in final_preds]
# final_preds_mean = np.mean(final_preds, axis=0)
# final_labels = (final_preds_mean >= 0.5).astype(int)
print(f'final aggregate accuracy BERT: {accuracy_score(final_preds_binary, ag_y)}')

# train = pg_rd,final_val = ag_pg: 0.6533333333333333
# train = pg_rd,final_val = ag_dm: 0.517
# train = pg_rd,final_val = ag_mt: 0.537
# jsut using 0s = 0.598

# train = pg_sr, final_val = ag_pg: 0.675
# train = pg_sr, final_val = ag_dm: 0.506
# train = pg_sr, final_val = ag_mt: 0.56

# train = pg_rs, final_val = ag_pg: 0.
# train = pg_rs, final_val = ag_dm: 0.
# train = pg_rs, final_val = ag_mt: 0.

# using both hyp and tgt, 
# train = pg_sr, final_val = ag_pg: 0.681
# train = pg_sr, final_val = ag_dm: 0.430
# train = pg_sr, final_val = ag_mt: 0.500

# train = dm_sr, final_val = ag_pg: 0.267
# train = dm_sr, final_val = ag_dm: 0.575
# train = dm_sr, final_val = ag_mt: 0.432

# train = mt_sr, final_val = ag_pg: 0.267
# train = mt_sr, final_val = ag_dm: 0.575
# train = mt_sr, final_val = ag_mt: 0.432

final aggregate accuracy BERT: 0.43238434163701067


In [ ]:
all_zeros = [0 for x in final_preds]
print(f'all zeros BERT: {accuracy_score(all_zeros, ag_y)}')

all zeros BERT: 0.597864768683274


In [ ]:
print(ag_y.value_counts())

label
0    336
1    226
Name: count, dtype: int64


In [ ]:
# i guess right now the big question is, is there a point in using cross validation?
# unless to get best hyper paramter, which i believe i can use some other library for that,
# what is the point? chatgpt said training final model invovles just using only
# train test split so...
final_preds = np.mean(all_val_preds, axis=0)
final_labels = (final_preds >= 0.5).astype(int)

print(f'Accuracy: {np.mean(accuracies):.4f}')
print(f'Precision: {np.mean(precisions):.4f}')
print(f'Recall: {np.mean(recalls):.4f}')
print(f'F1 Score: {np.mean(f1s):.4f}')
print(f'Final Predictions: {final_labels}')

Accuracy: 0.8427
Precision: 0.8756
Recall: 0.8427
F1 Score: 0.8171
Final Predictions: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0]


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.metrics import accuracy_score
import datasets

# Ensure eager execution
tf.config.run_functions_eagerly(True)

BATCH_SIZE = 8
# Create Pandas DataFrame without the index
final_df = pd.DataFrame({'hyp': ag_X, 'label': ag_y})

# Convert to HuggingFace Datasets
final_df2 = datasets.Dataset.from_pandas(final_df)
dataset = datasets.DatasetDict({'final': final_df2})

# Initialize tokenizer (make sure you have the right model name)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the datasets
def tokenize_function(example):
    return tokenizer(example['hyp'], truncation=True, padding='max_length', max_length=100)

tokenized_datasets_mapped = dataset.map(tokenize_function, batched=True, batch_size=BATCH_SIZE, remove_columns=['hyp','tgt'])

# Use from_generator to create dataset with correct output signature
# final_dataset = tf.data.Dataset.from_generator(
#     lambda: iter(tokenized_datasets_mapped['final']),
#     output_signature=(
#         {
#             'input_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32),
#             'attention_mask': tf.TensorSpec(shape=(None,), dtype=tf.int32),
#             'token_type_ids': tf.TensorSpec(shape=(None,), dtype=tf.int32)
#         },
#         tf.TensorSpec(shape=(), dtype=tf.int32)
#     )
# )
final_dataset = tf.data.Dataset.from_tensor_slices(tokenized_datasets_mapped['final'][:])
final_dataset = final_dataset.batch(BATCH_SIZE)
final_dataset = final_dataset.map(order_final, num_parallel_calls=tf.data.AUTOTUNE)

# Evaluate the model
final_preds = bert_model.predict(final_dataset).logits

# Apply sigmoid to get probabilities and flatten the tensor
final_preds = tf.nn.sigmoid(final_preds).numpy().squeeze()

# If final_preds is still multidimensional, reduce it to a 1D array
if final_preds.ndim > 1:
    final_preds = final_preds[:, 0]

# Calculate and print accuracy
accuracy = accuracy_score(ag_y, final_preds.round())
print(f'Final aggregate accuracy BERT: {accuracy}')

# keep track of results here for now
# for task pg, 0.645
# for task dm, 0.453
# for task mt, 0.486

Map: 100%|██████████| 375/375 [00:00<00:00, 1317.01 examples/s]


47/47 [==============================] - 61s 1s/step
Final aggregate accuracy BERT: 0.6426666666666667


In [ ]:
# import json
# copy_of_predictions = list(predictions)
# index = 0
# for i in range(len(copy_of_predictions)):
#     copy_of_predictions[i] = list(dict(copy_of_predictions[i]).values())
# #     for key,value in copy_of_predictions[i].items():
# #         copy_of_predictions[i][key] = list(value)

# print(copy_of_predictions)
# with open('mt_sr_160.json','w') as f:
# #     f.write(copy_of_predictions)
#     json.dump(copy_of_predictions, f)

In [ ]:
# kf.get_n_splits(X, y)